In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from typing import Sequence

from superprimitive_fusion.scanner import (
    capture_spherical_scans,
    render_rgb_view,
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
    triangulate_segments,
)

In [2]:
names = (
    ('mustard-bottle', 'mustard-bottle.obj'),
    ('table', 'table.obj'),
    # ('power-drill', 'power-drill.obj'),
    # ('bleach', 'bleach.obj'),
    # ('pitcher', 'pitcher.obj'),
    # ('mug', 'mug.obj'),
    # ('extra-large-clamp', 'extra-large-clamp-leaning.obj'),
)

meshes = dict()
for foldername,filename in names:
    print(f'Getting the {filename.split('.')[0]}')
    
    mesh = o3d.io.read_triangle_mesh(f"../data/posed-meshes/{foldername}/{filename}", enable_post_processing=True)

    bake_uv_to_vertex_colours(mesh)
    mesh.compute_vertex_normals()

    meshes[foldername] = mesh

meshlist = list(meshes.values())

Getting the mustard-bottle
Getting the table


In [3]:
centres = []
for meshname, mesh in meshes.items():
    if meshname == 'table':
        continue
    centres.append(mesh.get_center())

centres = np.vstack(centres)

obj_centre = centres.mean(axis=0)

In [4]:
def _quad_to_tris(idx: tuple[int, int, int, int],
                  verts: np.ndarray) -> tuple[list[int], list[int]]:
    """Split a quadrilateral into two triangles.

    The shortest diagonal is used for convex quads, while concave quads are
    triangulated so both resulting triangles are oriented counter‑clockwise
    (positive signed area).
    """
    def signed_area(a, b, c):
        return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0])

    # Convexity check – all cross products have the same sign
    crosses: list[float] = []
    for i in range(4):
        a, b, c = verts[i], verts[(i + 1) % 4], verts[(i + 2) % 4]
        ab, bc = b - a, c - b
        crosses.append(ab[0] * bc[1] - ab[1] * bc[0])
    is_convex = all(cp > 0 for cp in crosses)

    if is_convex:
        # pick the shorter diagonal
        d0 = np.sum((verts[0] - verts[2]) ** 2)
        d1 = np.sum((verts[1] - verts[3]) ** 2)
        if d0 <= d1:
            return [idx[0], idx[1], idx[2]], [idx[0], idx[2], idx[3]]
        return [idx[0], idx[1], idx[3]], [idx[1], idx[2], idx[3]]

    # Concave: test both diagonals for valid (positive‑area) triangles
    sa = [signed_area(*verts[[0, 1, 2]]), signed_area(*verts[[0, 2, 3]]),
          signed_area(*verts[[0, 1, 3]]), signed_area(*verts[[1, 2, 3]])]
    split1_ok = sa[0] > 0 and sa[1] > 0  # (0,1,2) & (0,2,3)
    split2_ok = sa[2] > 0 and sa[3] > 0  # (0,1,3) & (1,2,3)
    if split1_ok or not split2_ok:
        return [idx[0], idx[1], idx[2]], [idx[0], idx[2], idx[3]]
    return [idx[0], idx[1], idx[3]], [idx[1], idx[2], idx[3]]

def _interpolate_vertex_colors(
        mesh: o3d.geometry.TriangleMesh,
        primitive_ids: np.ndarray,
        bary_uv: np.ndarray
    ) -> np.ndarray:
    """Return RGB at hit points: sample texture if available, else barycentric vertex colors."""
    assert primitive_ids.ndim == 1 and bary_uv.ndim == 2, "Must not be img shape"

    # Texture path
    has_uvs = getattr(mesh, "has_triangle_uvs", lambda: False)()
    has_textures = hasattr(mesh, "textures") and len(mesh.textures) > 0
    if has_uvs and has_textures:
        # (3 * n_tris, 2) -> (n_tris, 3, 2)
        tri_uvs_all = np.asarray(mesh.triangle_uvs, dtype=np.float32).reshape(-1, 3, 2)
        tri_uvs = tri_uvs_all[primitive_ids]  # (N, 3, 2) for the hit triangles

        # Barycentric -> UV on the triangle
        u, v = bary_uv[:, 0], bary_uv[:, 1]
        w = 1.0 - u - v
        hit_uv = (tri_uvs[:, 0, :] * w[:, None] +
                  tri_uvs[:, 1, :] * u[:, None] +
                  tri_uvs[:, 2, :] * v[:, None]).astype(np.float32)

        # Choose texture per triangle if available; else use texture 0 for all.
        if hasattr(mesh, "triangle_material_ids") and len(mesh.textures) > 1:
            tri_mats_all = np.asarray(mesh.triangle_material_ids, dtype=np.int32)
            mat_ids = tri_mats_all[primitive_ids]
        else:
            mat_ids = np.zeros(len(hit_uv), dtype=np.int32)

        out = np.zeros((len(hit_uv), 3), dtype=np.float32)

        # Sample each texture group (bilinear)
        unique_tex = np.unique(mat_ids)
        for tex_id in unique_tex:
            mask = (mat_ids == tex_id)
            if not np.any(mask):
                continue

            img = np.asarray(mesh.textures[int(tex_id)])
            # Ensure H * W * C
            if img.ndim == 2:
                img = np.repeat(img[..., None], 3, axis=2)
            if img.shape[2] > 3:
                img = img[..., :3]  # drop alpha

            H, W = img.shape[0], img.shape[1]
            uv = np.clip(hit_uv[mask], 0.0, 1.0)
            x = uv[:, 0] * (W - 1)
            y = uv[:, 1] * (H - 1)

            x0 = np.floor(x).astype(np.int32)
            y0 = np.floor(y).astype(np.int32)
            x1 = np.clip(x0 + 1, 0, W - 1)
            y1 = np.clip(y0 + 1, 0, H - 1)

            wa = (x1 - x) * (y1 - y)
            wb = (x - x0) * (y1 - y)
            wc = (x1 - x) * (y - y0)
            wd = (x - x0) * (y - y0)

            Ia = img[y0, x0, :].astype(np.float32)
            Ib = img[y0, x1, :].astype(np.float32)
            Ic = img[y1, x0, :].astype(np.float32)
            Id = img[y1, x1, :].astype(np.float32)

            col = (Ia * wa[:, None] + Ib * wb[:, None] +
                   Ic * wc[:, None] + Id * wd[:, None]) / 255.0
            out[mask] = col

        return out

    # --- Fallback: per-vertex color interpolation ---
    vcols = np.asarray(mesh.vertex_colors, dtype=np.float32)
    tris = np.asarray(mesh.triangles, dtype=np.int32)[primitive_ids]
    c0, c1, c2 = vcols[tris[:, 0]], vcols[tris[:, 1]], vcols[tris[:, 2]]
    u, v = bary_uv[:, 0], bary_uv[:, 1]
    w = 1.0 - u - v
    return w[:, None] * c0 + u[:, None] * c1 + v[:, None] * c2


In [5]:
mesh: o3d.geometry.TriangleMesh = meshlist
cam_centre=polar2cartesian(1.0, 30, 90)
look_dir=obj_centre
dropout_rate: float = 0.0
depth_error_std: float = 0.0
translation_error_std: float = 0.0
rotation_error_std_degs: float = 0.0
width_px: int = 360
height_px: int = 240
fov: float = 70.0
k: float = 3.0

In [6]:
###############
# Ray casting #
###############
scene = o3d.t.geometry.RaycastingScene()
geom_ids = []
for name,mesh in meshes.items():
    id = scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))
    geom_ids.append(id)
    print(f'{name} is geom id {id}')

rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
    fov_deg=fov,
    center=list(look_dir),
    eye=list(cam_centre),
    up=[0, 0, 1],
    width_px=width_px,
    height_px=height_px,
)

ans = scene.cast_rays(rays)
t_hit = ans["t_hit"].numpy()

# Intersection metadata (gometry id + triangle id + barycentric uv + normals)
geom_ids = ans["geometry_ids"].numpy().astype(np.int16)
geom_ids[geom_ids > 1e2] = -1
prim_ids = ans["primitive_ids"].numpy().astype(np.int32)
bary_uv = ans.get("primitive_uvs", None).numpy()
normals = ans["primitive_normals"]

mustard-bottle is geom id 0
table is geom id 1


In [7]:
######################
# Dropout / validity #
######################
valid = np.isfinite(t_hit).reshape(-1)
# random dropout mask
n_dropout = int(dropout_rate * valid.size)
if n_dropout:
    dropout_idx = np.random.choice(valid.size, n_dropout, replace=False)
    valid[dropout_idx] = False

In [16]:
########################
# Generate 3‑D vertices#
########################
rays_np = rays.numpy()  # (H*W,6)
origins = rays_np[..., :3]
dirs = rays_np[..., 3:]
noise = (depth_error_std * np.random.randn(*t_hit.shape)).astype(np.float32)
t_noisy = t_hit + noise
verts = origins + dirs * t_noisy[..., None]

In [9]:
#############################
# Colour interpolation step #
#############################
vcols = np.full((*t_hit.shape, 3), 0.5)

# Assertions for bugfixing
assert bary_uv is not None
for mesh in meshlist:
    assert mesh.has_vertex_colors()

for id in range(len(meshlist)):
    rel_prim_ids = prim_ids[geom_ids==id]
    rel_bary_uv  =  bary_uv[geom_ids==id]
    vcols[geom_ids==id] = _interpolate_vertex_colors(meshlist[id], rel_prim_ids, rel_bary_uv)

In [10]:
def triangulate_rgbd_grid(verts, valid, z, obj_id=None, k=3.5,
                          normals=None, max_normal_angle_deg=None):
    """
    verts: (H*W,3) or (H,W,3)
    valid: (H*W,) or (H,W) bool
    z    : (H*W,) or (H,W) depth
    obj_id: (H*W,) or (H,W) int; unknown == -1
    normals: optional (H,W,3); if set, edges with normal jump > max_normal_angle_deg are cut
    """
    # reshape
    if verts.ndim == 2:
        N = verts.shape[0]
        H = int(np.round(np.sqrt(N)))
        W = N // H
        P = verts.reshape(H, W, 3)
    else:
        H, W, _ = verts.shape
        P = verts

    valid_img = valid.reshape(H, W)
    z_img = z.reshape(H, W)

    # disparity (robust jumps)
    disp = np.zeros_like(z_img, dtype=np.float32)
    m = valid_img & np.isfinite(z_img) & (z_img > 0)
    disp[m] = 1.0 / z_img[m]

    # object ids
    if obj_id is None:
        obj_id = np.full((H, W), -1, dtype=np.int32)
    else:
        obj_id = obj_id.reshape(H, W)

    # neighbor diffs
    dx = np.abs(disp[:, 1:] - disp[:, :-1])
    dy = np.abs(disp[1:, :] - disp[:-1, :])
    dd1 = np.abs(disp[:-1, :-1] - disp[1:, 1:])   # tl-br
    dd2 = np.abs(disp[:-1, 1:] - disp[1:, :-1])   # tr-bl

    mask_x  = valid_img[:, 1:] & valid_img[:, :-1]
    mask_y  = valid_img[1:, :] & valid_img[:-1, :]
    mask_d1 = valid_img[:-1, :-1] & valid_img[1:, 1:]
    mask_d2 = valid_img[:-1, 1:]  & valid_img[1:, :-1]

    vals = np.concatenate([
        dx[mask_x].ravel(), dy[mask_y].ravel(),
        (dd1[mask_d1] / np.sqrt(2)).ravel(),
        (dd2[mask_d2] / np.sqrt(2)).ravel()
    ])
    if vals.size:
        med = np.median(vals)
        mad = 1.4826 * np.median(np.abs(vals - med))
        base_thr = med + k * mad if mad > 0 else med * 1.5
    else:
        base_thr = np.inf

    thr_x, thr_y, thr_d = base_thr, base_thr, base_thr * np.sqrt(2)

    # same-object gates (unknown == -1 → cut)
    same_x  = (obj_id[:, 1:] == obj_id[:, :-1]) & (obj_id[:, 1:] >= 0)
    same_y  = (obj_id[1:, :] == obj_id[:-1, :]) & (obj_id[1:, :] >= 0)
    same_d1 = (obj_id[:-1, :-1] == obj_id[1:, 1:]) & (obj_id[:-1, :-1] >= 0)
    same_d2 = (obj_id[:-1, 1:]  == obj_id[1:, :-1]) & (obj_id[:-1, 1:]  >= 0)

    # base edge validity: depth, object, disparity
    good_x  = mask_x  & same_x  & (dx  <= thr_x)
    good_y  = mask_y  & same_y  & (dy  <= thr_y)
    good_d1 = mask_d1 & same_d1 & (dd1 <= thr_d)
    good_d2 = mask_d2 & same_d2 & (dd2 <= thr_d)

    # optional: normal-angle gate
    if (normals is not None) and (max_normal_angle_deg is not None):
        n = normals.reshape(H, W, 3).astype(np.float32)
        # normalize (cheap and safer; Open3D normals are ~unit but don’t rely on it)
        n_norm = np.linalg.norm(n, axis=-1, keepdims=True)
        ok = n_norm[..., 0] > 0
        n = np.where(ok[..., None], n / np.clip(n_norm, 1e-12, None), 0)

        cos_max = np.cos(np.deg2rad(max_normal_angle_deg))

        dot_x  = (n[:, 1:, :] * n[:, :-1, :]).sum(-1)
        dot_y  = (n[1:, :, :] * n[:-1, :, :]).sum(-1)
        dot_d1 = (n[:-1, :-1, :] * n[1:, 1:, :]).sum(-1)
        dot_d2 = (n[:-1, 1:, :]  * n[1:, :-1, :]).sum(-1)

        # require normals to be finite on both sides
        n_ok_x  = ok[:, 1:]  & ok[:, :-1]
        n_ok_y  = ok[1:, :]  & ok[:-1, :]
        n_ok_d1 = ok[:-1, :-1] & ok[1:, 1:]
        n_ok_d2 = ok[:-1, 1:]  & ok[1:, :-1]

        good_x  &= n_ok_x  & (dot_x  >= cos_max)
        good_y  &= n_ok_y  & (dot_y  >= cos_max)
        good_d1 &= n_ok_d1 & (dot_d1 >= cos_max)
        good_d2 &= n_ok_d2 & (dot_d2 >= cos_max)

    # indices per-quad
    id_img = np.arange(H * W, dtype=np.int32).reshape(H, W)
    tl = id_img[:-1, :-1]; tr = id_img[:-1, 1:]
    bl = id_img[1:, :-1];  br = id_img[1:, 1:]

    # per-quad edge masks
    e_top    = good_x[:H-1, :W-1]
    e_bottom = good_x[1:,  :W-1]
    e_left   = good_y[:H-1, :W-1]
    e_right  = good_y[:H-1, 1:]
    d1 = good_d1
    d2 = good_d2

    # triangles for each diagonal
    A = e_top & e_right & d1            # (tl,tr,br)
    B = e_left & e_bottom & d1          # (tl,br,bl)
    C = e_top & e_left & d2             # (tl,tr,bl)
    D = e_right & e_bottom & d2         # (tr,br,bl)

    # counts per orientation
    count1 = A.astype(np.uint8) + B.astype(np.uint8)
    count2 = C.astype(np.uint8) + D.astype(np.uint8)

    # tie-break by shorter 3D diagonal; compute only where endpoints are finite (no warnings)
    d1_len2 = np.full((H-1, W-1), np.inf, dtype=P.dtype)
    d2_len2 = np.full((H-1, W-1), np.inf, dtype=P.dtype)

    fin_d1 = np.isfinite(P[:-1, :-1, :]).all(-1) & np.isfinite(P[1:, 1:, :]).all(-1)
    if fin_d1.any():
        diff = P[:-1, :-1, :][fin_d1] - P[1:, 1:, :][fin_d1]
        d1_len2[fin_d1] = (diff * diff).sum(-1)

    fin_d2 = np.isfinite(P[:-1, 1:, :]).all(-1) & np.isfinite(P[1:, :-1, :]).all(-1)
    if fin_d2.any():
        diff = P[:-1, 1:, :][fin_d2] - P[1:, :-1, :][fin_d2]
        d2_len2[fin_d2] = (diff * diff).sum(-1)

    prefer_d1 = (count1 > count2) | ((count1 == count2) & (d1_len2 <= d2_len2))

    A &= prefer_d1
    B &= prefer_d1
    C &= ~prefer_d1
    D &= ~prefer_d1

    # pack tris
    tris = []
    def add(mask, i0, i1, i2):
        if mask.any():
            tris.append(np.stack([i0[mask], i1[mask], i2[mask]], axis=1))
    add(A, tl, tr, br)
    add(B, tl, br, bl)
    add(C, tl, tr, bl)
    add(D, tr, br, bl)

    if not tris:
        return np.empty((0, 3), dtype=np.int32)
    return np.concatenate(tris, axis=0).astype(np.int32)


In [17]:
tris = triangulate_rgbd_grid(
    verts=verts,
    valid=valid,
    z=t_hit,
    obj_id=geom_ids,
    k=3.5,
    normals=None,                   # or your per-pixel normals (H,W,3)
    max_normal_angle_deg=None       # e.g. 60 to be stricter on folds
).tolist()

In [20]:
mesh_out = o3d.geometry.TriangleMesh()
mesh_out.vertices = o3d.utility.Vector3dVector(verts.reshape(-1, 3))
mesh_out.triangles = o3d.utility.Vector3iVector(np.asarray(tris, dtype=np.int32)[:, [0,2,1]])

if vcols is not None:
    mesh_out.vertex_colors = o3d.utility.Vector3dVector(vcols.reshape(-1,3))

#############################
# Clean‑up / post‑processing#
#############################
mesh_out.remove_unreferenced_vertices()
mesh_out.remove_degenerate_triangles()
mesh_out.remove_duplicated_triangles()
mesh_out.remove_non_manifold_edges()
mesh_out.compute_vertex_normals()

######################
# Registration error #
######################
mesh_out.translate(tuple(np.random.randn(3) * translation_error_std))
R = mesh.get_rotation_matrix_from_xyz(
    tuple(np.random.randn(3) * np.deg2rad(rotation_error_std_degs))
)
mesh_out.rotate(R, center=mesh_out.get_center())

mesh = mesh_out

In [21]:
o3d.visualization.draw_geometries([mesh])